# Externally Computed Polars

This example shows how to build, using the level-2 interface, an aviary model that includes an external susbsystem that computes a lift and drag polar and passes them into the mission aerodynamics for a 3-phase mission (climb, cruise, descent). During the mission, Aviary will interpolate on the computed polars to compute actual lift and drag for a given flight condition.

We start with the assumption that we have an external component called {glue:md}`ExternalAero` that can compute the lift and drag at any given altitude, Mach number, and angle of attack. The details of such a component may be highly complicated and not important for the purposes of this example. We will be using a structured grid, which assumes the data table is regularly spaced in all dimensions. We want to compute lift and drag over a grid of altitudes (in 'ft'), Mach numbers, and angles of attack given by:

In [1]:
# This hidden cell just creates a component that produces the lift and drag polar.
# The implementation details are unimportant to the example.
import numpy as np
import openmdao.api as om

import aviary.api as av
from aviary.api import Aircraft

# The drag-polar-generating component reads this in, instead of computing the polars.
polar_file = 'models/large_single_aisle_1/large_single_aisle_1_aero_free_reduced_alpha.csv'

aero_data, _, _ = av.read_data_file(
    polar_file,
    aliases={
        'altitude': 'altitude',
        'mach': 'mach',
        'angle_of_attack': 'angle_of_attack',
        'lift_coefficient': 'cl',
        'drag_coefficient': 'cd',
    },
)

altitude = np.unique(aero_data.get_val('altitude', 'ft'))
mach = np.unique(aero_data.get_val('mach', 'unitless'))
angle_of_attack = np.unique(aero_data.get_val('angle_of_attack', 'deg'))

shape = (altitude.size, mach.size, angle_of_attack.size)
CL = aero_data.get_val('lift_coefficient').reshape(shape)
CD = aero_data.get_val('drag_coefficient').reshape(shape)


class ExternalAero(om.ExplicitComponent):
    """
    This component is a stand-in for an externally computed lift/drag table
    calculation. It does nothing but read in the pre-computed table. A real
    component would actually computed the values at all requested points.
    """

    def initialize(self):
        """Declare options."""
        self.options.declare(
            'altitude', default=None, allow_none=True, desc='List of altitudes in ascending order.'
        )
        self.options.declare(
            'mach', default=None, allow_none=True, desc='List of Mach numbers in ascending order.'
        )
        self.options.declare(
            'angle_of_attack',
            default=None,
            allow_none=True,
            desc='List of angles of attack in ascending order.',
        )

    def setup(self):
        altitude = self.options['altitude']
        mach = self.options['mach']
        angle_of_attack = self.options['angle_of_attack']

        self.add_input(Aircraft.Wing.AREA, 1.0, units='ft**2')
        self.add_input(Aircraft.Wing.SPAN, 1.0, units='ft')

        shape = (len(altitude), len(mach), len(angle_of_attack))

        self.add_output('drag_table', shape=shape, units='unitless')
        self.add_output('lift_table', shape=shape, units='unitless')

    def compute(self, inputs, outputs):
        """
        This component doesn't do anything, except set the drag and lift
        polars from the file we read in.
        """
        # Your component will compute CD and CL for a grid of altitudes, machs, and
        # angles of attack, and return them in a multidimensional array as described
        # in the example text.

        # Because it would be prohibitive to embed something like a vortex lattice
        # code in this example, we are "cheating" here by sending a pre-computed
        # drag polar.

        outputs['drag_table'] = CD
        outputs['lift_table'] = CL


print('Altitude (ft)', altitude)
print('Mach', mach)
print('Angle of Attack (deg)', angle_of_attack)

Altitude (ft) [    0.  3000.  6000.  9000. 12000. 15000. 18000. 21000. 24000. 27000.
 30000. 33000. 36000. 38000. 42000.]
Mach [0.   0.2  0.4  0.5  0.6  0.7  0.75 0.8  0.85 0.9 ]
Angle of Attack (deg) [-2.  0.  2.  4.  6.  8. 10.]


In [2]:
# Testing Cell
from aviary.utils.doctape import glue_variable

# class ExternalAero is defined within this file
glue_variable(ExternalAero.__name__, md_code=True)

/home0/jkirk5/python/Aviary/aviary/visualization/dashboard.py:60: UserWarning: Using Panel interactively in VSCode notebooks requires the jupyter_bokeh package to be installed. You can install it with:

   pip install jupyter_bokeh

or:
    conda install jupyter_bokeh

and try again.
  pn.extension(sizing_mode='stretch_width')


/home0/jkirk5/python/Aviary/aviary/visualization/dashboard.py:62: UserWarning: Using Panel interactively in VSCode notebooks requires the jupyter_bokeh package to be installed. You can install it with:

   pip install jupyter_bokeh

or:
    conda install jupyter_bokeh

and try again.
  pn.extension('tabulator')


`ExternalAero`

In a structured grid, interpolation data must be present for every combination of inputs. In other words, our {glue:md}`ExternalAero` component must run a full factorial of points spanning those 3 variables. The Aviary variable hierarchy includes two variables for the polars: {glue:md}`Aircraft.Design.LIFT_POLAR`, and {glue:md}`Aircraft.Design.DRAG_POLAR`. The data in each of these polars should be a `n` x `m` x `k` numpy array, where `n` is the number of altitudes, `m` is the number of mach numbers, and `k` is the number of angles of attack. The `ExternalAero` will need to compute these values and place them into an array of this shape.

If use of a structured grid is not desirable, then the data does not need to meet these formatting requirements. In that case, the data table does not have to be regularly spaced, and each variable (`Altitude`, `Mach`, `angle_of_attack`, `LIFT_POLAR`, and `DRAG_POLAR`) must be 1-dimensional numpy arrays of equal length.

Using the level-2 interface, we create a builder for our external `ExternalAero` subsystem. In this example, the component produces outputs `drag_table` and `lift_table`, but we can specify an alias to {glue:md}`Aircraft.Design.DRAG_POLAR` and {glue:md}`Aircraft.Design.LIFT_POLAR` respectively. It is important that we inherit from the {glue:md}`AerodyanmicsBuilder` to let Aviary know this is builder produces aerodynamics components. Some mission analysis methods require special handling of aerodynamics components that will not occur if we skip this step.

In [3]:
# Testing Cell
from aviary.interface.methods_for_level2 import AviaryProblem
from aviary.subsystems.aerodynamics.aerodynamics_builder import AerodyanmicsBuilder
from aviary.utils.doctape import get_variable_name, glue_variable

glue_variable(AerodyanmicsBuilder.__name__, md_code=True)
glue_variable(get_variable_name(Aircraft.Design.LIFT_POLAR), md_code=True)
glue_variable(get_variable_name(Aircraft.Design.DRAG_POLAR), md_code=True)
glue_variable(AviaryProblem.__name__, md_code=False)

`AerodyanmicsBuilder`

`Aircraft.Design.LIFT_POLAR`

`Aircraft.Design.DRAG_POLAR`

AviaryProblem

In [4]:
class ExternalAeroBuilder(av.AerodyanmicsBuilder):
    """
    An example subsystem builder that adds an external aerodynamics component.

    Parameters
    ----------
    aero_data : NamedValues
        Altitude, Mach number, and angle of attack data, all in ascending order.
    """

    def __init__(self, name='aero', altitude=None, mach=None, angle_of_attack=None):
        super().__init__(name)
        self.altitude = altitude
        self.mach = mach
        self.angle_of_attack = angle_of_attack

    def build_pre_mission(self, aviary_inputs):
        """
        Build an OpenMDAO system for the pre-mission computations of the subsystem.

        Returns
        -------
        pre_mission_sys : openmdao.core.Group
            An OpenMDAO group containing all computations that need to happen in
            the pre-mission part of the Aviary problem. This includes sizing, design,
            and other non-mission parameters.
        """
        aero_group = om.Group()
        aero = ExternalAero(
            altitude=self.altitude, mach=self.mach, angle_of_attack=self.angle_of_attack
        )
        aero_group.add_subsystem(
            'premission_aero',
            aero,
            promotes_inputs=['*'],
            promotes_outputs=[
                ('drag_table', Aircraft.Design.DRAG_POLAR),
                ('lift_table', Aircraft.Design.LIFT_POLAR),
            ],
        )
        return aero_group

Notice that we have passed the altitude, Mach, and angle of attack arrays into the builder so that the ExternalAero component can use them as instantiation arguments.

Next, the existing mission phases need to be given the information they need to set up our aerodynamics analysis using _glue:md}`phase_info`. We use the `tabular_cruise` method with the `solve_alpha` flag of Aviary's included aerodynamics for this, which can accept the input passed from our external subsystem. Since we are using Aviary's built-in aerodynamics methods, we use the default name "aerodynamics". Don't forget to update the `subsystem_options` for each phase. We must specify the `method`, the `aero_data` that contains our altitude, Mach, and angle of attack data, as well as the `connect_training_data` flag to denote we are passing our drag polars via openMDAO connections.

In [5]:
subsystem_options = {
    'method': 'tabular_cruise',
    'solve_alpha': True,
    'aero_data': aero_data,
    'connect_training_data': True,
}

phase_info = av.default_height_energy_phase_info.copy()
phase_info['climb']['subsystem_options'] = {'aerodynamics': subsystem_options}
phase_info['cruise']['subsystem_options'] = {'aerodynamics': subsystem_options}
phase_info['descent']['subsystem_options'] = {'aerodynamics': subsystem_options}

Finally, we can instantiate the {glue:md}`AviaryProblem`, and add our external subsystem. This problem requires an extra step, where we must tell Aviary the size of our lift and drag polars so that it can allocate the right shape for the connection.

In [6]:
from aviary.utils.functions import get_aviary_resource_path

input_file = get_aviary_resource_path('models/aircraft/test_aircraft/aircraft_for_bench_GwFm.csv')

external_aero = ExternalAeroBuilder(
    name='external_aero', altitude=altitude, mach=mach, angle_of_attack=angle_of_attack
)

prob = av.AviaryProblem()
prob.load_inputs(input_file, phase_info)
prob.load_external_subsystems([external_aero])
# Preprocess inputs
prob.check_and_preprocess_inputs()

# Add correctly-sized polar to aviary_inputs so that input components are sized correctly.
shape = (altitude.size, mach.size, angle_of_attack.size)

prob.aviary_inputs.set_val(Aircraft.Design.LIFT_POLAR, np.zeros(shape), units='unitless')
prob.aviary_inputs.set_val(Aircraft.Design.DRAG_POLAR, np.zeros(shape), units='unitless')

# get through setup for the problem
prob.add_pre_mission_systems()
prob.add_phases()
prob.add_post_mission_systems()
prob.link_phases()
prob.add_driver('SLSQP')
prob.add_design_variables()
prob.add_objective(objective_type='mass', ref=-1e5)
prob.setup()
prob.set_initial_guesses()

prob.run_model()


pre_mission.core_subsystems.mass.wing_mass.isolated_mass
NL: Newton 0 ; 2102.07591 1
NL: Newton 1 ; 0.104452174 4.96900108e-05
NL: Newton 2 ; 2.51020538e-10 1.19415544e-13
NL: Newton Converged

pre_mission.core_subsystems.mass.fuel_mass
NL: Newton 0 ; 53148.6766 1
NL: Newton 1 ; 227.052065 0.00427201727
NL: Newton 2 ; 0.00792420681 1.49095092e-07
NL: Newton 3 ; 7.29015462e-12 1.37165308e-16
NL: Newton Converged

traj.phases.climb.indep_states
NL: Newton Converged in 0 iterations

traj.phases.climb.rhs_all.solver_sub
+  
+  =================================================
+  traj.phases.climb.rhs_all.solver_sub.aerodynamics
+  =================================================
+  NL: Newton 0 ; 3.3187881e-06 1
+  NL: Newton 1 ; 5.56886169e-10 0.000167798049
+  NL: Newton Converged
NL: Newton 0 ; 0.0741907181 1
+  
+  =================================================
+  traj.phases.climb.rhs_all.solver_sub.aerodynamics
+  =================================================
+  NL: Newton 0

In [ ]:
# Testing Cell
# Make sure we successfully passed the polar
from openmdao.utils.assert_utils import assert_near_equal

om_CD = prob.get_val(Aircraft.Design.DRAG_POLAR)[0, 0, 0]
assert_near_equal(om_CD, CD[0, 0, 0], 1e-6)

0.0